# Tasks 

## Task 1

### creating the function for the bit rotation to the right

In [20]:
# Function for roating left 
def rotl(x, n=1, bits=32):
    return ((x << n) & (2**bits - 1)) | (x >> (bits - n))

In [21]:
# Function for roating right
def rotr(x, n=1, bits=32):
    return (x >> n) | ((x << (bits - n)) & (2**bits - 1))

In [22]:
# number 
numL = 1001001101
numL2 = 0b00010000111101010101010101010101

# imuting into funciton 
#rotae_num = rotl(num2, 1, 32)
rotae_numL = rotl(numL2, 1, 32)
print(bin(rotae_numL))


0b100001111010101010101010101010


In [23]:
## test case by chatGPT ( https://chatgpt.com/share/67a34032-81f4-800e-8a7b-f8011aadb398 )
num = 0b1001001101  # Binary representation (589 in decimal)
rotated_num = rotl(num, 1, 32)  # Rotate left by 1 bit
print(bin(rotated_num))  # Should show the properly rotated value

0b10010011010


In [24]:
numR= 0b00010000111101010101010101010101
rotae_numR=rotr(numR, 1, 32)
print(bin(rotae_numR))

0b10001000011110101010101010101010


In [25]:
# 3 valuse x,y,z
x= 0b1001001101
y= 0b00010000111101010101010101010101
z= 0b11010000111101010101010101010101
# x is the sector x(1) = y, x(0) = z


## Task 2

## Task 3

## Task 4

## Task 5

## Task 6

## Task 7